In [1]:
# https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators

In [2]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

In [3]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [4]:
df = pd.read_excel('raw data/WDIEXCEL.xlsx')


In [5]:
def read_wb(df):
    df.drop(columns='Indicator Code', inplace=True)
    df['Data Source'] = 'World Bank'
    df['Unit'] = 'metric tons'
    df['Metric'] = 'Annual production'
    for x in df['Indicator Name']:
        if x=='Aquaculture production (metric tons)':
            df.replace(x, 'Aquaculture Production', inplace=True)
        elif x=='Capture fisheries production (metric tons)':
            df.replace(x, 'Capture Fisheries', inplace=True)
    df.rename(columns={'Indicator Name': 'Technology Name'}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.iloc[48:]
    df.reset_index(drop=True, inplace=True)
    df.drop(40, inplace=True)
    iso = []
    for country in df['Country Name']:
        if country == 'World':
            iso2 = 'World'
        else:
            iso2 = coco.convert(names=country, to='iso2')
        iso.append(iso2)
    df.drop(columns='Country Code', inplace=True)
    df['Country Code'] = iso
    type_list = []
    for x in range(217):
        name = df['Country Name'].iloc[x]
        if name != 'World':
            type_list.append('National')
        else:
            type_list.append('Global')
    df['Spatial Scale'] = type_list
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    omit = []
    for col in df.columns:
        if len(col) != 4 or col == 'Unit':
            omit.append(col)
    empty_rows = df.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    na_idx = []
    for country in df.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    df.drop(na_idx, inplace=True)
    return df

In [6]:
aqua = df[df['Indicator Name']=='Aquaculture production (metric tons)']
aqua = read_wb(aqua)

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='Indicator Code', inplace=True)
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data Source'] = 'World Bank'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [7]:
cap_fish = df[df['Indicator Name']=='Capture fisheries production (metric tons)']
cap_fish = read_wb(cap_fish)

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='Indicator Code', inplace=True)
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data Source'] = 'World Bank'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_16868/172142170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [8]:
wb = pd.concat([aqua, cap_fish])

In [9]:
wb.to_csv('cleaned data/wb.csv', index=False)